In [2]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 16.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=06794111597a47cfe4a74926f1c0b4f8ac1bd7e8624514e6e9d32f8babec05a7
  Stored in directory: /Users/margaritamikaelan/Library/Caches/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum as spark_sum

In [4]:
spark = SparkSession.builder.appName("GoIT Data Analysis").getOrCreate()

24/11/24 15:18:18 WARN Utils: Your hostname, Margaritas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.152 instead (on interface en0)
24/11/24 15:18:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/24 15:18:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
purchases_df = spark.read.csv(
    "/Users/margaritamikaelan/Documents/goit/goit-de-hw-03/purchases (1).csv",
    header=True,
    inferSchema=True,
)
products_df = spark.read.csv(
    "/Users/margaritamikaelan/Documents/goit/goit-de-hw-03/products (1).csv",
    header=True,
    inferSchema=True,
)
users_df = spark.read.csv(
    "/Users/margaritamikaelan/Documents/goit/goit-de-hw-03/users (1).csv",
    header=True,
    inferSchema=True,
)

In [6]:
# 2. Очищення даних (видалення рядків із пропущеними значеннями)
purchases_df_clean = purchases_df.dropna()
products_df_clean = products_df.dropna()
users_df_clean = users_df.dropna()

In [9]:
# 3. Підрахунок кількості проданих одиниць товарів за категоріями
total_quantity_per_category = purchases_with_products.groupBy("category").agg(
    spark_sum("quantity").alias("total_quantity")
)

print("Кількість проданих одиниць за категоріями:")
total_quantity_per_category.show()

Кількість проданих одиниць за категоріями:
+-----------+--------------+
|   category|total_quantity|
+-----------+--------------+
|       Home|           197|
|     Sports|           296|
|Electronics|           218|
|   Clothing|           152|
|     Beauty|            58|
+-----------+--------------+



In [10]:
# 4. Підрахунок суми покупок за категоріями для вікової категорії 18-25
users_filtered = users_df_clean.filter((col("age") >= 18) & (col("age") <= 25))

purchases_with_users = purchases_with_products.join(users_filtered, "user_id")


total_quantity_per_category_18_25 = purchases_with_users.groupBy("category").agg(
    spark_sum("quantity").alias("total_quantity_18_25")
)

print("Кількість проданих одиниць за категоріями для вікової категорії 18-25:")
total_quantity_per_category_18_25.show()

Кількість проданих одиниць за категоріями для вікової категорії 18-25:
+-----------+--------------------+
|   category|total_quantity_18_25|
+-----------+--------------------+
|       Home|                  42|
|     Sports|                  53|
|Electronics|                  31|
|   Clothing|                  45|
|     Beauty|                   5|
+-----------+--------------------+



In [13]:
# 5. Підрахунок загальної суми всіх покупок для вікової категорії 18–25

total_spent_18_25 = (
    purchases_with_users.withColumn("total_spent", col("quantity") * col("price"))
    .groupBy("category")
    .agg(spark_sum("total_spent").alias("total_spent_18_25"))
)


total_spent_18_25_sum = total_spent_18_25.select(
    spark_sum("total_spent_18_25").alias("total_sum")
).collect()[0]["total_sum"]


spent_percentage = total_spent_18_25.withColumn(
    "total_spent_18_25", round(col("total_spent_18_25"), 2)
).withColumn(
    "percentage", round((col("total_spent_18_25") / total_spent_18_25_sum) * 100, 2)
)


print("Частка покупок за кожною категорією:")
spent_percentage.show()

Частка покупок за кожною категорією:
+-----------+-----------------+----------+
|   category|total_spent_18_25|percentage|
+-----------+-----------------+----------+
|       Home|            361.1|      29.9|
|     Sports|            310.5|     25.71|
|Electronics|            249.6|     20.67|
|   Clothing|            245.0|     20.29|
|     Beauty|             41.4|      3.43|
+-----------+-----------------+----------+



In [15]:
# Отримання топ-3 категорій за часткою витрат для вікової категорії 18-25
top_3_categories = spent_percentage.orderBy(col("percentage").desc()).limit(3)

print("Топ-3 категорії за часткою витрат (18-25 років):")
top_3_categories.show()

Топ-3 категорії за часткою витрат (18-25 років):
+-----------+-----------------+----------+
|   category|total_spent_18_25|percentage|
+-----------+-----------------+----------+
|       Home|            361.1|      29.9|
|     Sports|            310.5|     25.71|
|Electronics|            249.6|     20.67|
+-----------+-----------------+----------+

